# <span style='color:red'>Project 2.  Due October 23</span>

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

### In this project we develop a first-order algorithm to construct a portfolio using intraday data.

In [29]:
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import datetime

##### We will have data involving $n$ assets, and use the first $T$ days of the data to compute the portfolio.
##### The computation will produce a weight $x_i$ for each asset $i = 1,...,n$, which could be long or short.
##### We assume that on each day, a position is taken at the open, and closed at noon.  So we define:
$$ p^o_{j,t} = \ \text{price of asset $j$ on day $t$ at the open}$$
$$ p^1_{j,t} = \ \text{price of asset $j$ on day $t$ at noon}$$
$$ r_{j,t} =  \ \frac{p^1_{j,t} - p^o_{j,t}}{p^o_{j,t}} = \ \text{return earned by asset $j$ on day $t$.}$$
$$ \bar r_j = \ \frac{1}{T} \sum_{t = 1}^T r_{j,t} = \ \text{average return earned by asset $j$.}$$

#### The optimization problem to solve depends on two parameters: $\theta \ge 0$ and $\pi > 0 0$.
####
$$ \text{minimize} \ \left(-\sum_{j = 1}^n \bar r_j x_j\right) \ + \ \theta \left( \frac{1}{T} \sum_{t = 1}^{T}\left[\sum_{j = 1}^n (r_{j,t} -  \bar r_j)x_j\right]^\pi\right)^{1/\pi}$$
#### 
#### There are no constraints on the quantities $x_j$.
#### The first sum is minus the average return earned by the portfolio.  In the second sum, the quantity inside the square brackets is the excess return earned by the portfolio on day $t$, magnified by the power $\pi$.  The quantity $\theta$ is a risk aversion parameter.
 

In [93]:
def OCfilter(dt: datetime.datetime):
    return dt.time() == datetime.time(9,30) or dt.time() == datetime.time(12, 00)

def dailyReturn(df):
    dt = pd.to_datetime(df['Dates'])
    filt = dt.apply(OCfilter)
    df = df.loc[filt,'Close'].reset_index(drop=True).pct_change()
    df = df[df.index%2==1]
    return df

def csvProcess(filename):
    df = pd.read_csv(filename, low_memory=False,header=3)
    df.loc[0,'Dates'] = df.loc[1,'Dates'].replace('31','30')
    numactual = df.iloc[:, 6].astype(float).isna().argmax()
    df = df.iloc[:numactual,:3]
    df = dailyReturn(df)
    return df

def objective(x, ret, average, pi, theta):
    term1 = -np.sum(average * x)
    inner_sum = np.sum((ret - average) * x, axis=1)
    term2 = np.power(np.mean(np.power(np.abs(inner_sum), pi)), 1/pi)
    return term1 + theta * term2

filename=['AMZN.csv','NFLX.csv','TSLA.csv']

T = 100
# Store daily return of 3 assets for first T days
returnarray = np.zeros((T,3))
for i in range(len(filename)):
    returnarray[:,i] = csvProcess(filename[i])[:T]

# Store average return for 3 assets for first T days
average = np.zeros(3)
for i in range(3):
    average[i] = np.mean(returnarray[:,i])
display(average,returnarray)

# Initialize weight
portfolio = [1 / 3, 1 / 3, 1 / 3]

print('Obejective value test: '+ str(objective(portfolio,returnarray,average,0.5,0.1)))


array([ 0.00033531, -0.00021261, -0.00071473])

array([[-2.71392669e-02,  8.63236670e-03,  1.80581875e-02],
       [ 6.99256646e-03,  2.50697924e-02,  1.62621242e-02],
       [ 1.06732948e-02, -1.17606365e-02,  1.63345355e-02],
       [ 1.03029536e-02, -1.93530550e-03,  2.93487498e-02],
       [-5.05077558e-03, -3.13390933e-03,  2.61749238e-02],
       [ 2.07119700e-03, -5.18960630e-03, -2.12734437e-03],
       [-6.00192061e-03,  1.39606654e-02,  3.11620056e-02],
       [ 1.64062375e-02, -9.38454436e-03, -2.29971608e-02],
       [-4.77545857e-03, -1.46062247e-02,  8.13550264e-03],
       [-8.08332881e-03, -9.42857143e-03, -1.90932461e-02],
       [-1.96905857e-03, -1.36706629e-03,  9.99207073e-04],
       [ 1.78145960e-02, -1.60403498e-02, -1.08334789e-02],
       [ 9.96914817e-03, -1.16297072e-02, -6.81750294e-03],
       [ 2.98132271e-03, -1.01714724e-02, -2.03091774e-04],
       [-1.31927704e-02, -1.96239674e-04,  2.06966739e-02],
       [ 3.03385172e-03,  9.96910051e-03, -5.41619798e-03],
       [-1.71075811e-02,  2.00980483e-02

Obejective value test: 0.0007668345284937486


### <span style='color:red'> Task 1. Develop a first-order method to address this computational problem.</span>
#### 
#### Your method should work with values of $T$ at least $100$. Use the data we provide for AMZN, NFLX, TSLA, i.e., $n = 3$. 
###
#### Make sure your code works with $\pi = 0.5, 2, 4, 6$, and $\theta = 0.1, 10, 1000, 10^5, 10^6$.

### <span style='color:red'>Task 2: Benchmark your portfolio on the remaining days</span>
#### On each of the remaining days, we proceed as follows.  Denote by $x^*$ your portfolio. At the market open we invest $10^9 x^*_j$ on each asset $j$, and we close the position (by) noon.  You need to use the asset's price to compute the number of shares that you invest in, whether long or short. So the total you invest equals $$ \sum_{j = 1}^n 10^9 |x^*_j|.$$
#### Report the average return earned by your portfolio.